In [1]:
!pip install openpyxl

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [24]:
from modelscope import snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer

# Downloading model checkpoint to a local dir model_dir
# model_dir = snapshot_download('qwen/Qwen-7B')
# model_dir = snapshot_download('qwen/Qwen-7B-Chat')
# model_dir = snapshot_download('qwen/Qwen-14B')
model_dir = snapshot_download('qwen/Qwen-1_8B-Chat-Int4', revision='master')
print(model_dir)

# Loading local checkpoints
# trust_remote_code is still set as True since we still load codes from local dir instead of transformers
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto",
    trust_remote_code=True
).eval()

2024-02-17 09:55:23,843 - modelscope - WARNING - Using the master branch is fragile, please use it with caution!
2024-02-17 09:55:23,844 - modelscope - INFO - Use user-specified model revision: master


/mnt/workspace/.cache/modelscope/qwen/Qwen-1_8B-Chat-Int4


TypeError: QWenLMHeadModel.__init__() got an unexpected keyword argument 'disable_exllama'

In [32]:
with open('city.txt','r',encoding='utf-8') as fp:
    city_list=fp.readlines()
    city_list=[line.strip().split(' ')[1] for line in city_list]

In [33]:
Q='青岛4月6日下雨么?'

prompt_template='''
给定一句话：“%s”，请你按步骤要求工作。

步骤1：识别这句话中的城市和日期共2个信息
步骤2：根据城市和日期信息，生成JSON字符串，格式为{"city":城市,"date":日期}

请问，这个JSON字符串是：
'''

In [39]:
import random
import json
import time 

Q_arr=[]
A_arr=[]

Q_list=[
    ('{city}{year}年{month}月{day}日的天气','%Y-%m-%d'),
    ('{city}{year}年{month}月{day}号的天气','%Y-%m-%d'),
    ('{city}{month}月{day}日的天气','%m-%d'),
    ('{city}{month}月{day}号的天气','%m-%d'),

    ('{year}年{month}月{day}日{city}的天气','%Y-%m-%d'),
    ('{year}年{month}月{day}号{city}的天气','%Y-%m-%d'),
    ('{month}月{day}日{city}的天气','%m-%d'),
    ('{month}月{day}号{city}的天气','%m-%d'),

    ('你们{year}年{month}月{day}日去{city}玩吗？','%Y-%m-%d'),
    ('你们{year}年{month}月{day}号去{city}玩么？','%Y-%m-%d'),
    ('你们{month}月{day}日去{city}玩吗？','%m-%d'),
    ('你们{month}月{day}号去{city}玩吗？','%m-%d'),
]

# 生成一批"1月2号"、"1月2日"、"2023年1月2号", "2023年1月2日", "2023-02-02", "03-02"之类的话术, 教会它做日期转换
for i in range(1000):
    Q=Q_list[random.randint(0,len(Q_list)-1)]
    city=city_list[random.randint(0,len(city_list)-1)]
    year=random.randint(1990,2025)
    month=random.randint(1,12)
    day=random.randint(1,28)
    time_str='{}-{}-{}'.format(year,month,day)
    date_field=time.strftime(Q[1],time.strptime(time_str,'%Y-%m-%d'))
    Q=Q[0].format(city=city,year=year,month=month,day=day) # 问题
    A=json.dumps({'city':city,'date':date_field},ensure_ascii=False)  # 回答

    Q_arr.append(prompt_template%(Q,))
    A_arr.append(A)

train_data = []
for i in range(len(Q_arr)):
    example = {
        'id': 'identity_{}'.format(i),
        'conversations': [
            {
                'from': 'user',
                'value': Q_arr[i],
            },
            {
                'from': 'assistant',
                'value': A_arr[i]
            }
        ]
    }
    train_data.append(example)

with open('train.txt', 'w', encoding='utf-8') as fp:
    fp.write(json.dumps(train_data))

微调模型，生成到output_qwen

In [40]:
!bash finetune/finetune_lora_single_gpu.sh -m /mnt/workspace/.cache/modelscope/qwen/Qwen-1_8B-Chat-Int4 -d /mnt/workspace/train.txt

[2024-02-17 10:35:21,335] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2024-02-17 10:35:22.749612: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-17 10:35:22.751932: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-17 10:35:22.781846: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-17 10:35:22.781864: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-17 10:35:22.781885:

加载SFT后的模型

In [41]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    'output_qwen', # path to the output directory
    device_map="auto",
    trust_remote_code=True
).eval()

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
Try importing flash-attention for faster inference...


In [42]:
model.generation_config.top_p=0 # 只选择概率最高的token

Q_list=['2020年4月16号三亚下雨么？','青岛3-15号天气预报','5月6号下雪么，城市是威海','青岛2023年12月30号有雾霾么?','我打算6月1号去北京旅游，请问天气怎么样？','你们打算1月3号坐哪一趟航班去上海？','小明和小红是8月8号在上海结婚么?',
        '一起去东北看冰雕么，大概是1月15号左右，我们3个人一起']
for Q in Q_list:
    prompt=prompt_template%(Q,)
    A,hist=model.chat(tokenizer,prompt,history=None)
    print('Q:%s\nA:%s\n'%(Q,A))

Q:2020年4月16号三亚下雨么？
A:{"city": "三亚", "date": "2020-04-16"}

Q:青岛3-15号天气预报
A:{"city": "青岛", "date": "03-15"}

Q:5月6号下雪么，城市是威海
A:{"city": "威海", "date": "05-06"}

Q:青岛2023年12月30号有雾霾么?
A:{"city": "青岛", "date": "2023-12-30"}

Q:我打算6月1号去北京旅游，请问天气怎么样？
A:{"city": "北京", "date": "06-01"}

Q:你们打算1月3号坐哪一趟航班去上海？
A:{"city": "上海", "date": "01-03"}

Q:小明和小红是8月8号在上海结婚么?
A:{"city": "上海", "date": "08-08"}

Q:一起去东北看冰雕么，大概是1月15号左右，我们3个人一起
A:{"city": "东北", "date": "01-15"}



In [43]:
prompt='青岛海边钓鱼需要特别注意什么？'
resp,hist=model.chat(tokenizer,prompt,history=None)
print(resp)

1. 避免在恶劣天气下钓鱼，如雷雨、大风等。
2. 遵守当地的法律法规和规定，不要破坏海洋环境。
3. 不要携带违禁物品上岸，以免被罚款或扣押证件。
